<a href="https://colab.research.google.com/github/ravi-gopalan/DAND_Data_Wrangling/blob/master/custom_entity_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [2]:
cd '/gdrive/My Drive/abv_reviews'

/gdrive/My Drive/abv_reviews


In [3]:
ls -l

total 101761
-rw------- 1 root root     2000 Dec 26 18:06  bread_keyword_patterns.npz
-rw------- 1 root root      589 Dec 26 17:27  breads_list_2.jsonl
-rw------- 1 root root    26438 Dec 26 16:44  breads_list.jsonl
-rw------- 1 root root    25359 Dec 15 03:21  CoherenceScore.csv
-rw------- 1 root root     4968 Dec 26 18:06  condiment_keyword_patterns.npz
-rw------- 1 root root     2353 Dec 26 16:29  condiments_list_2.jsonl
-rw------- 1 root root    43320 Dec 26 16:21  condiments_list.jsonl
-rw------- 1 root root  8428475 Dec 21 13:01  datalist_corrected.csv
-rw------- 1 root root    24864 Dec 26 17:05  df_check_breads.csv
-rw------- 1 root root    40909 Dec 26 16:21  df_check_condiments.csv
-rw------- 1 root root   185178 Dec 26 08:09  df_check_de.csv
-rw------- 1 root root  4580904 Dec 26 08:10  df_check_en.csv
-rw------- 1 root root    56341 Dec 26 09:17  df_check_es.csv
-rw------- 1 root root    43595 Dec 26 09:17  df_check_fr.csv
-rw------- 1 root root    72573 Dec 26 17:05  df_ch

In [4]:
!pip install spacymoji
from spacymoji import Emoji

     |████████████████████████████████| 51kB 2.9MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42175 sha256=df802941bf1a613e2d3b0a23a2bf9848488ea85891cbc1405bebf06747d56b54
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [0]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

import random

import spacy
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
from spacy import displacy
from spacy.util import minibatch, compounding
from pandas.io.json import json_normalize

In [6]:
import spacy.cli
spacy.cli.download("en_core_web_lg")
import en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
from spacy.gold import docs_to_json
from spacy.gold import biluo_tags_from_offsets
from spacy.gold import offsets_from_biluo_tags

In [0]:
import re

In [9]:
df_pattern = pd.read_json('patterns.jsonl',lines=True)
df_pattern.head()

,label,pattern,id
0,dish,[{'LOWER': ''mpanatigghi'}],'mpanatigghi
1,dish,[{'LOWER': ''nduja'}],'nduja
2,dish,"[{'LOWER': ''ota'}, {'LOWER': ''ika'}]",'ota_'ika
3,dish,"[{'LOWER': '.amaro'}, {'LOWER': 'ramazzotti'}]",.amaro_ramazzotti
4,dish,"[{'LOWER': '15'}, {'LOWER': 'bean'}, {'LOWER':...",15_bean_soup


In [0]:

def cleanse_text(text):
  step1 = re.sub(r"(\{'LOWER':\s)",r"",str(text))
  step2 = re.sub(r"[\[{\'\,\"}\]]",r"",step1)
  return step2



In [11]:
df_pattern['cleaned_pattern'] = df_pattern.pattern.apply(lambda x: cleanse_text(x))
df_pattern['label'] = 'U_' + df_pattern.label.astype(str)
df_pattern.head()

,label,pattern,id,cleaned_pattern
0,U_dish,[{'LOWER': ''mpanatigghi'}],'mpanatigghi,mpanatigghi
1,U_dish,[{'LOWER': ''nduja'}],'nduja,nduja
2,U_dish,"[{'LOWER': ''ota'}, {'LOWER': ''ika'}]",'ota_'ika,ota ika
3,U_dish,"[{'LOWER': '.amaro'}, {'LOWER': 'ramazzotti'}]",.amaro_ramazzotti,.amaro ramazzotti
4,U_dish,"[{'LOWER': '15'}, {'LOWER': 'bean'}, {'LOWER':...",15_bean_soup,15 bean soup


In [12]:
df_pattern['tup_col'] = list(zip(df_pattern.cleaned_pattern, df_pattern.label))
df_pattern['merged'] = df_pattern.apply(lambda row: {row['cleaned_pattern']:row['label']}, axis=1)
df_pattern.head()



,label,pattern,id,cleaned_pattern,tup_col,merged
0,U_dish,[{'LOWER': ''mpanatigghi'}],'mpanatigghi,mpanatigghi,"(mpanatigghi, U_dish)",{'mpanatigghi': 'U_dish'}
1,U_dish,[{'LOWER': ''nduja'}],'nduja,nduja,"(nduja, U_dish)",{'nduja': 'U_dish'}
2,U_dish,"[{'LOWER': ''ota'}, {'LOWER': ''ika'}]",'ota_'ika,ota ika,"(ota ika, U_dish)",{'ota ika': 'U_dish'}
3,U_dish,"[{'LOWER': '.amaro'}, {'LOWER': 'ramazzotti'}]",.amaro_ramazzotti,.amaro ramazzotti,"(.amaro ramazzotti, U_dish)",{'.amaro ramazzotti': 'U_dish'}
4,U_dish,"[{'LOWER': '15'}, {'LOWER': 'bean'}, {'LOWER':...",15_bean_soup,15 bean soup,"(15 bean soup, U_dish)",{'15 bean soup': 'U_dish'}


In [13]:

ent_list = df_pattern.tup_col.to_list()
ent_list[50:60]

[('agliata', 'U_dish'),
 ('agneau du périgord', 'U_dish'),
 ('agnolotti', 'U_dish'),
 ('agrodolce', 'U_dish'),
 ('agua de sevilla', 'U_dish'),
 ('agua de valencia', 'U_dish'),
 ('aguachile', 'U_dish'),
 ('aguadito de pollo', 'U_dish'),
 ('aguapanela', 'U_dish'),
 ('aguas frescas', 'U_dish')]

In [0]:
nlp = spacy.load("en_core_web_lg")
ruler = EntityRuler(nlp).from_disk("patterns.jsonl")
emoji = Emoji(nlp)
nlp.add_pipe(emoji, first=True)
nlp.add_pipe(ruler, before='ner')

In [16]:
ruler.patterns[10:15]

[{'id': '5_hour_energy',
  'label': 'dish',
  'pattern': [{'LOWER': '5'}, {'LOWER': 'hour'}, {'LOWER': 'energy'}]},
 {'id': '7_and_7',
  'label': 'dish',
  'pattern': [{'LOWER': '7'}, {'LOWER': 'and'}, {'LOWER': '7'}]},
 {'id': 'a_gei',
  'label': 'dish',
  'pattern': [{'LOWER': 'a'}, {'LOWER': 'gei'}]},
 {'id': 'a_thoke',
  'label': 'dish',
  'pattern': [{'LOWER': 'a'}, {'LOWER': 'thoke'}]},
 {'id': 'aachener_printen',
  'label': 'dish',
  'pattern': [{'LOWER': 'aachener'}, {'LOWER': 'printen'}]}]

In [17]:
nlp.pipe_names

['emoji', 'tagger', 'parser', 'entity_ruler', 'ner']

In [18]:
ruler.patterns[6200]

{'id': 'litsea_cubeba',
 'label': 'spice',
 'pattern': [{'LOWER': 'litsea'}, {'LOWER': 'cubeba'}]}

In [15]:
doc1 = nlp("I love fettuccine cheese 💚🌿 but hate buffalo wings.")
print([(ent.text, ent.label_, ent.ent_id_) for ent in doc1.ents])

displacy.render(doc1,'ent',jupyter=True)



[('fettuccine', 'dish', 'fettuccine'), ('buffalo wings', 'dish', 'buffalo_wings')]


In [16]:
doc2 = nlp("Avocado 🌿 are good but the ajwain is great")
displacy.render(doc2,'ent',jupyter=True)

In [21]:
nlp.pipe_names

['emoji', 'tagger', 'parser', 'entity_ruler', 'ner']

In [17]:
doc3 = nlp("Baguette is the best bread in the whole world - bagels come close and ciabatta is also up there")
displacy.render(doc3,'ent',jupyter=True)

In [18]:
nlp.disable_pipes('entity_ruler')

[('entity_ruler', <spacy.pipeline.entityruler.EntityRuler at 0x7f5fdaccc630>)]

In [19]:
doc3 = nlp("Baguette is the best chaunk bread in the whole world - bagels come close and ciabatta is also up there")
displacy.render(doc3,'ent',jupyter=True)

In [20]:
reviews = pd.read_csv('reviews_text.csv',usecols=['_id', 'text'])
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56156 entries, 0 to 56155
Data columns (total 2 columns):
_id     56156 non-null object
text    56155 non-null object
dtypes: object(2)
memory usage: 877.6+ KB


In [21]:
reviews.dropna(inplace=True)
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56155 entries, 0 to 56155
Data columns (total 2 columns):
_id     56155 non-null object
text    56155 non-null object
dtypes: object(2)
memory usage: 1.3+ MB


In [22]:
texts = reviews['text'][0:1000]
texts

0      Burger joint offers a wide range of cheeseburg...
1      It was really good. The mushroom broth was esp...
2      $8.90 for sesame rice, mushroom rendang, curry...
3      2 mains + 1 green rice bento from greendot. Re...
4      The lion mane mushroom rendang is so delicious...
                             ...                        
995    This dish is 6-spoon appetizer made of tea lea...
996    These churros are so simple, original, and cri...
997    Warm fluffy soft bread with avocado and tofu f...
998    The avo + tofu didn’t fly well together but th...
999    Super original and definitely worth trying if ...
Name: text, Length: 1000, dtype: object

In [0]:
ingredients_list = ['pistachio', 'almond', 'walnut', 'cashew', 'peanuts', 'macadamia', 'hazelnuts', 'pecan', 'brazil nut', 'pine nut' ]
issues_list = ['crumble - maybe a verb', 'milk']
food_times = 
to_be_added_list = ['spirulina', 'herbs', 'micro-greens', 'greens','spinach','eggs', 'nut butters', 'cream cheese', 'cake', 'pepper', 'zucchini', 'aubergines',  'tomato',\
                    'oysters', 'kale', 'cucumber', 'quinoa', 'tomatoes', 'onion' ,'garlic', 'avocado', 'chilli', 'dark chocolate', 'mushrooms', 'coriander' ,'corn chips',\
                    'marinara', 'donuts', 'olives', 'crackers', 'waffles', 'strawberry', 'blueberry', 'raspberry', 'compote', 'mayo', 'mayonnaise', 'soy milk', 'oat milk', \
                    'coconut milk', 'almond milk', 'caffeine', 'milk tea', 'cauliflower', 'matcha', 'sorbet', 'banana', 'goji berry', 'chickpea', 'spinach', 'romaine',\
                    'arugula', 'cacao', 'Portobello', 'pear', 'plant milk', 'BBQ sauce', 'artichoke', 'mango', 'sandwich', 'capsicum', 'wedge', 'kang kong', 'seitan',\
                    'ciabatta', 'focaccia', 'bean sprouts', 'olive oil']

In [27]:
check = np.random.randint(0,50000,3)
print(check)

for doc in list(nlp.pipe(texts[check])):
  displacy.render(doc,'ent',jupyter=True)
#  displacy.render(doc,'dep',jupyter=True,options={'distance': 60, 'fine_grained': True})



[49671 32259 46827]


In [28]:

displacy.render(nlp("Very fresh and juicy coconut FRUIT Lassi DISH . Didn't thought it would be so fresh"),'dep',jupyter=True)

In [29]:
doc = nlp("I like London. I also like Bangalore")
tok_list = [token for token in doc]
print(tok_list, len(tok_list))


[I, like, London, ., I, also, like, Bangalore] 8


In [30]:
tags = ['O' for tok in tok_list]
tags

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [0]:
tags[2] = 'U-LOC'
tags[7] = 'U-LOC'
tags

['O', 'O', 'U-LOC', 'O', 'O', 'O', 'O', 'U-LOC']

In [0]:
entities = offsets_from_biluo_tags(doc, tags)
entities

[(7, 13, 'LOC'), (27, 36, 'LOC')]

In [31]:
doc = nlp("I love Punjabi cuisine.")
[ent.label_ for ent in doc.ents]

['NORP']

['NORP']

In [32]:
tok_list = [token for token in doc]
print(tok_list, len(tok_list))
tags = ['O' for tok in tok_list]
print(tags)

[I, love, Punjabi, cuisine, .] 5
['O', 'O', 'O', 'O', 'O']


In [0]:
tags[2] = 'U-NORP'
print(tags)
entities = offsets_from_biluo_tags(doc, tags)
print(entities)


['O', 'O', 'U-NORP', 'O', 'O']
[(7, 15, 'NORP')]


['O', 'O', 'U-LOC', 'O']

In [0]:
json_data = docs_to_json([doc])
json_data['paragraphs'][0]['sentences'][0]['tokens'][2]['ner']

'U-NORP'

In [0]:
import srsly

In [0]:
f = open('rice.txt')
docs = [] # initialize a list to be populated wih strings
lines = f.readlines()  # returns a list of srings from the txt file
for line in lines:
    # print(line[:]) # display the sentence from that line
    doc = nlp(line) # convert string into a spacy doc object using nlp
    docs.append(doc) # add new doc to the list of docs

json_data = docs_to_json(docs) # convert doc into a json file
srsly.write_json('spacy_train.json', [json_data])


In [0]:
f.close()

In [48]:
type(ent_list + check_list)
[i[0] for i in ent_list + check_list]

['mpanatigghi',
 'nduja',
 'ota ika',
 '.amaro ramazzotti',
 '15 bean soup',
 '1519 tequila',
 '2007 vietnam food scare',
 '20th century',
 '3 a.m. vodka',
 '4 copas',
 '5 hour energy',
 '7 and 7',
 'a gei',
 'a thoke',
 'aachener printen',
 'aam panna',
 'aamras',
 'abbaye de citeaux',
 'abbaye de tamié',
 'abertam cheese',
 'abgoosht',
 'abondance',
 'aborrajado',
 'abricotine',
 'absinthe',
 'aburaage',
 'acar',
 'acarajé',
 'acceglio (cheese)',
 'accelerade',
 'achar',
 'acini di pepe',
 'ackee and saltfish',
 'acqua pazza',
 'acquacotta',
 'acquasale',
 'adana kebabı',
 'adjika',
 'adobo',
 'afelia',
 'affogato',
 'afghan biscuit',
 'afritada',
 'afternoon snack',
 'afuegal pitu',
 'agedashi dofu',
 'agemochi',
 'agent orange',
 'aggala',
 'aglianico',
 'agliata',
 'agneau du périgord',
 'agnolotti',
 'agrodolce',
 'agua de sevilla',
 'agua de valencia',
 'aguachile',
 'aguadito de pollo',
 'aguapanela',
 'aguas frescas',
 'agwijjim',
 'ahlgrens bilar',
 'ahtapot salata',
 'aioli'

In [0]:
check_list = [('bok','U_vegetable'),('choy','U_vegetable'),\
              ('bean','U_vegetable'), ('sprouts','U_vegetable'),\
              ('soffritto','U_dish'),('patties','U_dish'),('cakes','U_dish'),('parmesan','U_dish'),('cheese','U_dish'),\
              ('soy','U_condiment'),('sauce','U_condiment'),\
              ('olive','U_condiment'),('oil','U_condiment'),\
              ('wine','U_beverage'),\
              ('3/4-inch','U_QUANTITY'), ('2-inch','U_QUANTITY'),('3-inch','U_QUANTITY'),('6-inch','U_QUANTITY'),\
              ('first','U_CARDINAL'),('second','U_CARDINAL'),('third','U_CARDINAL'),('fourth','U_CARDINAL'),('fifth','U_CARDINAL'),('last','U_CARDINAL'),\
              ('1','U_ORDINAL'),('2','U_ORDINAL'),('3','U_ORDINAL'),('4','U_ORDINAL'),('5','U_ORDINAL'),('6','U_ORDINAL'),('10','U_ORDINAL'),\
              ('100','U_ORDINAL'),('one','U_ORDINAL'),('two','U_ORDINAL'),('three','U_ORDINAL'),('four','U_ORDINAL'),('five','U_ORDINAL'),('six','U_ORDINAL'),\
              ('seven','U_ORDINAL'),('eight','U_ORDINAL'),('nine','U_ORDINAL'),('ten','U_ORDINAL'),\
              ('minute','U_TIME'),('minutes','U_TIME'),('seconds','U_TIME'),('hour','U_TIME'),('hours','U_TIME'),('day','U_TIME')]

In [64]:
''.join(list(set([tup[1] for tup in ent_list + check_list if tup[0] == 'rice'])))

'U_dish'

In [0]:
f = open('rice.txt')
docs = [] # initialize a list to be populated wih strings
lines = f.readlines()  # returns a list of srings from the txt file
for line in lines:
    # print(line[:]) # display the sentence from that line
    doc = nlp(line) # convert string into a spacy doc object using nlp
    docs.append(doc) # add new doc to the list of docs
f.close()



In [0]:

train_gen = (nlp(line) for line in open('rice.txt'))
type(train_gen)
to_train_ents = []

In [27]:
l = next(train_gen)
tok_list = [token.text for token in l] 
print(l, tok_list)
tag_list = ['O' for token in l]
print(tag_list)
matched_ents = [(i,item) for i, item in enumerate(tok_list) if item.lower() in [i[0] for i in ent_list + check_list]] 
print(matched_ents)

for ent in matched_ents:
  ent_tag = ''.join(list(set([tup[1] for tup in ent_list + check_list if tup[0] == ent[1].lower()])))
  print(ent[0], ent[1], ent_tag)
  tag_list[ent[0]] = ent_tag

print(tag_list)
entities = offsets_from_biluo_tags(l, tag_list)
print(entities)
to_train_ents.append((l, dict(entities = entities)))
print(l, entities)  

Cook the rice as you normally would, but be careful not to use too much water. The rice should not be too sticky. It may help to let the cooked rice sit overnight in the refrigerator.
 ['Cook', 'the', 'rice', 'as', 'you', 'normally', 'would', ',', 'but', 'be', 'careful', 'not', 'to', 'use', 'too', 'much', 'water', '.', 'The', 'rice', 'should', 'not', 'be', 'too', 'sticky', '.', 'It', 'may', 'help', 'to', 'let', 'the', 'cooked', 'rice', 'sit', 'overnight', 'in', 'the', 'refrigerator', '.', '\n']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[(2, 'rice'), (19, 'rice'), (33, 'rice')]
2 rice U_dish
19 rice U_dish
33 rice U_dish
['O', 'O', 'U_dish', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'U_dish', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'U_dish', 'O', 'O', 'O', 'O', 'O

In [67]:
to_train_ents

[(If using tofu, cook it., {'entities': [(9, 13, 'dish')]}),
 (Cook the rice as you normally would, but be careful not to use too much water. The rice should not be too sticky. It may help to let the cooked rice sit overnight in the refrigerator.,
  {'entities': [(9, 13, 'dish'), (83, 87, 'dish'), (144, 148, 'dish')]}),
 (If using onions, dice them into whatever size you prefer.,
  {'entities': [(9, 15, 'vegetable')]}),
 (If using bok choy, separate the stalk part from the leaf part. Slice the leaf part crosswise into 3/4-inch-thick strips, then tear the strips into 2-inch pieces. These pieces will be treated much like the bean sprouts, though perhaps with slightly more cooking. The stalk part will be treated as celery.,
  {'entities': [(9, 12, 'vegetable'),
    (13, 17, 'vegetable'),
    (98, 106, 'QUANTITY'),
    (147, 153, 'QUANTITY'),
    (205, 209, 'vegetable'),
    (210, 217, 'vegetable'),
    (296, 302, 'vegetable')]}),
 (If using bean sprouts, wash them and discard any that are

In [82]:
to_train_ents = []

for _, doc in enumerate(docs):
  print(_)
  tok_list = [token.text for token in doc] 
#  print(tok_list)
  tag_list = ['O' for token in doc]
#  print(tag_list)
  matched_ents = [(i,item) for i, item in enumerate(tok_list) if item.lower() in [i[0] for i in ent_list + check_list]] 
#  print(matched_ents)
  print(doc)
  for ent in matched_ents:
    ent_tag = ''.join(list(set([tup[1] for tup in ent_list + check_list if tup[0] == ent[1].lower()])))
    print(ent[0], ent[1], ent_tag)
    tag_list[ent[0]] = ent_tag
  
#  print(tag_list)
  entities = offsets_from_biluo_tags(doc, tag_list)
#  print(entities)
  to_train_ents.append((doc, dict(entities = entities)))
  print(doc, entities)

0
If using tofu, cook it.

2 tofu U_dish
If using tofu, cook it.
 [(9, 13, 'dish')]
1
Cook the rice as you normally would, but be careful not to use too much water. The rice should not be too sticky. It may help to let the cooked rice sit overnight in the refrigerator.

2 rice U_dish
19 rice U_dish
33 rice U_dish
Cook the rice as you normally would, but be careful not to use too much water. The rice should not be too sticky. It may help to let the cooked rice sit overnight in the refrigerator.
 [(9, 13, 'dish'), (83, 87, 'dish'), (144, 148, 'dish')]
2
If using onions, dice them into whatever size you prefer.

2 onions U_vegetable
If using onions, dice them into whatever size you prefer.
 [(9, 15, 'vegetable')]
3
If using bok choy, separate the stalk part from the leaf part. Slice the leaf part crosswise into 3/4-inch-thick strips, then tear the strips into 2-inch pieces. These pieces will be treated much like the bean sprouts, though perhaps with slightly more cooking. The stalk part w

In [75]:
to_train_ents

[(If using tofu, cook it., {'entities': [(9, 13, 'dish')]}),
 (Cook the rice as you normally would, but be careful not to use too much water. The rice should not be too sticky. It may help to let the cooked rice sit overnight in the refrigerator.,
  {'entities': [(9, 13, 'dish'), (83, 87, 'dish'), (144, 148, 'dish')]}),
 (If using onions, dice them into whatever size you prefer.,
  {'entities': [(9, 15, 'vegetable')]}),
 (If using bok choy, separate the stalk part from the leaf part. Slice the leaf part crosswise into 3/4-inch-thick strips, then tear the strips into 2-inch pieces. These pieces will be treated much like the bean sprouts, though perhaps with slightly more cooking. The stalk part will be treated as celery.,
  {'entities': [(9, 12, 'vegetable'),
    (13, 17, 'vegetable'),
    (98, 106, 'QUANTITY'),
    (147, 153, 'QUANTITY'),
    (205, 209, 'vegetable'),
    (210, 217, 'vegetable'),
    (296, 302, 'vegetable')]}),
 (If using bean sprouts, wash them and discard any that are

In [0]:
all_tok_list = []
all_ent_tag_list = []
nlp_doc = []
with open ('rice.txt') as train_file:
  doc = True
  while doc:
    doc = train_file.readline()
    tok_list = [token for token in nlp(doc)]
    nlp_doc.append(nlp(doc))
    print(tok_list)
    all_tok_list.append(tok_list)
    ent_tag_list = ['O' for tok in tok_list]
    print(ent_tag_list)
    all_ent_tag_list.append(ent_tag_list)



[If, using, tofu, ,, cook, it, ., 
]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[Cook, the, rice, as, you, normally, would, ,, but, be, careful, not, to, use, too, much, water, ., The, rice, should, not, be, too, sticky, ., It, may, help, to, let, the, cooked, rice, sit, overnight, in, the, refrigerator, ., 
]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[If, using, onions, ,, dice, them, into, whatever, size, you, prefer, ., 
]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[If, using, bok, choy, ,, separate, the, stalk, part, from, the, leaf, part, ., Slice, the, leaf, part, crosswise, into, 3/4-inch, -, thick, strips, ,, then, tear, the, strips, into, 2-inch, pieces, ., These, pieces, will, be, treated, much, like, the, bean, sprouts, ,, though, perhaps, with, slightly, more, cooking, ., The, stalk, part, wi

In [0]:
all_ent_tag_list[0][2] = 'U-dish'
all_ent_tag_list[1][2] = 'U-dish'
all_ent_tag_list[1][19] = 'U-dish'
all_ent_tag_list[1][33] = 'U-dish'
all_ent_tag_list[2][2] = 'U-vegetable'
all_ent_tag_list[3][2] = 'B-vegetable'
all_ent_tag_list[3][3] = 'L-vegetable'
all_ent_tag_list[3][41] = 'B-vegetable'
all_ent_tag_list[3][42] = 'L-vegetable'
all_ent_tag_list[3][58] = 'U-vegetable'
all_ent_tag_list[4][2] = 'B-vegetable'
all_ent_tag_list[4][3] = 'L-vegetable'
all_ent_tag_list[6][28] = 'U-dish'
all_ent_tag_list[7][2] = 'U-dish'

all_ent_tag_list[8][2] = 'B-condiment'
all_ent_tag_list[8][3] = 'L-condiment'

all_ent_tag_list[8][8] = 'U-dish'
all_ent_tag_list[8][16] = 'U-vegetable'
all_ent_tag_list[8][18] = 'B-vegetable'
all_ent_tag_list[8][19] = 'L-vegetable'

all_ent_tag_list[9][1] = 'U-condiment'

all_ent_tag_list[10][2] = 'U-dish'

all_ent_tag_list[10][4] = 'B-vegetable'
all_ent_tag_list[10][5] = 'L-vegetable'

all_ent_tag_list[10][8] = 'U-vegetable'
all_ent_tag_list[10][11] = 'U-dish'
all_ent_tag_list[12][48] = 'U-dish'

all_ent_tag_list[14][3] = 'U-vegetable'
all_ent_tag_list[14][24] = 'B-condiment'
all_ent_tag_list[14][25] = 'L-condiment'
all_ent_tag_list[14][32] = 'U-dish'
all_ent_tag_list[14][34] = 'U-GPE'

all_ent_tag_list[15][2] = 'U-dish'
all_ent_tag_list[15][23] = 'U-condiment'
all_ent_tag_list[16][32] = 'U-dish'
all_ent_tag_list[16][35] = 'U-dish'

all_ent_tag_list[17][21] = 'U-dish'
all_ent_tag_list[17][43] = 'U-dish'

all_ent_tag_list[17][68] = 'U-CARDINAL'
all_ent_tag_list[17][70] = 'U-CARDINAL'
all_ent_tag_list[17][71] = 'U-TIME'

all_ent_tag_list[18][5] = 'U-dish'
all_ent_tag_list[18][10] = 'U-dish'
all_ent_tag_list[18][27] = 'U-dish'
all_ent_tag_list[18][34] = 'U-dish'
all_ent_tag_list[19][30] = 'U-dish'
all_ent_tag_list[21][6] = 'U-TIME'

all_ent_tag_list[22][4] = 'U-condiment'
all_ent_tag_list[22][18] = 'U-condiment'
all_ent_tag_list[22][26] = 'U-condiment'

all_ent_tag_list[23][3] = 'U-dish'
all_ent_tag_list[24][1] = 'U-dish'
all_ent_tag_list[24][21] = 'U-dish'

all_ent_tag_list[25][6] = 'U-dish'
all_ent_tag_list[25][18] = 'U-dish'
all_ent_tag_list[25][27] = 'U-dish'
all_ent_tag_list[25][69] = 'U-vegetable'
all_ent_tag_list[25][71] = 'U-vegetable'
all_ent_tag_list[25][87] = 'U-dish'
all_ent_tag_list[25][102] = 'U-dish'
all_ent_tag_list[25][113] = 'U-dish'
all_ent_tag_list[25][160] = 'U-dish'

all_ent_tag_list[25][143] = 'U-dish'
all_ent_tag_list[26][5] = 'U-dish'
all_ent_tag_list[26][23] = 'U-dish'
all_ent_tag_list[27][10] = 'U-condiment'
all_ent_tag_list[29][2] = 'U-dish'
all_ent_tag_list[29][31] = 'U-dish'
all_ent_tag_list[31][0] = 'U-spice'
all_ent_tag_list[31][6] = 'U-dish'

all_ent_tag_list[32][10] = 'U-vegetable'
all_ent_tag_list[32][18] = 'U-vegetable'
all_ent_tag_list[32][21] = 'U-vegetable'

all_ent_tag_list[33][6] = 'B-dish'
all_ent_tag_list[33][7] = 'L-dish'

all_ent_tag_list[33][17] = 'U-dish'
all_ent_tag_list[33][26] = 'U-dish'
all_ent_tag_list[33][29] = 'U-CARDINAL'
all_ent_tag_list[33][30] = 'U-TIME'
all_ent_tag_list[34][10] = 'U-dish'
all_ent_tag_list[34][2] = 'U-ORDINAL'
all_ent_tag_list[35][2] = 'U-dish'
all_ent_tag_list[35][25] = 'U-dish'
all_ent_tag_list[35][40] = 'B-condiment'
all_ent_tag_list[35][41] = 'L-condiment'
all_ent_tag_list[35][27] = 'U-dish'
all_ent_tag_list[35][45] = 'U-dish'
all_ent_tag_list[35][44] = 'U-dish'
all_ent_tag_list[36][15] = 'U-dish'
all_ent_tag_list[36][17] = 'U-ORDINAL'
all_ent_tag_list[36][18] = 'U-TIME'
all_ent_tag_list[38][23] = 'U-dish'
all_ent_tag_list[38][38] = 'U-dish'
all_ent_tag_list[38][29] = 'U-ORDINAL'
all_ent_tag_list[38][34] = 'U-ORDINAL'
all_ent_tag_list[39][2] = 'U-CARDINAL'
all_ent_tag_list[39][11] = 'U-condiment'
all_ent_tag_list[40][5] = 'U-CARDINAL'
all_ent_tag_list[40][2] = 'U-condiment'
all_ent_tag_list[40][13] = 'U-dish'
all_ent_tag_list[40][27] = 'U-condiment'
all_ent_tag_list[41][18] = 'U-dish'
all_ent_tag_list[41][36] = 'U-dish'
all_ent_tag_list[42][3] = 'U-ORDINAL'
all_ent_tag_list[42][5] = 'U-ORDINAL'
all_ent_tag_list[42][6] = 'U-TIME'


In [0]:
check_list = []
ent_list.extend(check_list)
ent_list

["'mpanatigghi",
 "'nduja",
 "'ota 'ika",
 '.amaro ramazzotti',
 '15 bean soup',
 '1519 tequila',
 '2007 vietnam food scare',
 '20th century',
 '3 a.m. vodka',
 '4 copas',
 '5 hour energy',
 '7 and 7',
 'a gei',
 'a thoke',
 'aachener printen',
 'aam panna',
 'aamras',
 'abbaye de citeaux',
 'abbaye de tamié',
 'abertam cheese',
 'abgoosht',
 'abondance',
 'aborrajado',
 'abricotine',
 'absinthe',
 'aburaage',
 'acar',
 'acarajé',
 'acceglio (cheese)',
 'accelerade',
 'achar',
 'acini di pepe',
 'ackee and saltfish',
 'acorn squash',
 'acqua pazza',
 'acquacotta',
 'acquasale',
 'adana kebabı',
 'adjika',
 'adjika',
 'adobo',
 'advieh',
 'afelia',
 'affogato',
 'afghan biscuit',
 'afritada',
 'afternoon snack',
 "afuega'l pitu",
 'agedashi dofu',
 'agemochi',
 'agent orange',
 'aggala',
 'aglianico',
 'agliata',
 'agneau du périgord',
 'agnolotti',
 'agrodolce',
 'agua de sevilla',
 'agua de valencia',
 'aguachile',
 'aguadito de pollo',
 'aguapanela',
 'aguas frescas',
 'agwijjim',
 '

In [0]:
[(i,item.text) for i, item in enumerate(all_tok_list[42]) if item.text.lower() in ent_list.extend(check_list)]

In [0]:
[(i,item.text) for i, item in enumerate(all_tok_list[42]) if item.text.lower() in ['rice','sausage','bok','choy','bean','sprouts','celery','soy','sauce', 'eggs','onions', 'pepper', 'sugar', 'tofu','onion',\
                                                                           'risotto', 'rissotto','olive', 'oil','soffritto','Italy','15','20','minutes','minute','butter','herb','potatoes','broccoli','mushrooms',\
                                                                           'saffron','pumpkin','leek', 'parmesan','cheese','first','second','third','last','3','cakes','ten','one','two','three','four','five','six','seven']]

[(3, 'five'), (5, 'seven'), (6, 'minutes')]

In [0]:
all_ent_tag_list

[['O', 'O', 'U-DISH', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'U-DISH',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'U-DISH',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'U-DISH',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O', 'O', 'U-DISH', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'B-VEGETABLE',
  'L-VEGETABLE',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-VEGETABLE',
  'L-VEGETABLE',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'U-VEGETABLE',
  'O',
  'O'],
 ['O',
  'O',
  'B-VEGETABLE',
  'L-VEGETABLE',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
 

In [0]:
all_entities = []
for i in range(len(all_tok_list)):
  print(all_tok_list[i])
  print(all_ent_tag_list[i])
  doc = nlp_doc[i]
  entities = offsets_from_biluo_tags(doc, all_ent_tag_list[i])
  all_entities.append(entities)


[If, using, tofu, ,, cook, it, ., 
]
['O', 'O', 'U-dish', 'O', 'O', 'O', 'O', 'O']
[Cook, the, rice, as, you, normally, would, ,, but, be, careful, not, to, use, too, much, water, ., The, rice, should, not, be, too, sticky, ., It, may, help, to, let, the, cooked, rice, sit, overnight, in, the, refrigerator, ., 
]
['O', 'O', 'U-dish', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'U-dish', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'U-dish', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[If, using, onions, ,, dice, them, into, whatever, size, you, prefer, ., 
]
['O', 'O', 'U-vegetable', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[If, using, bok, choy, ,, separate, the, stalk, part, from, the, leaf, part, ., Slice, the, leaf, part, crosswise, into, 3/4-inch, -, thick, strips, ,, then, tear, the, strips, into, 2-inch, pieces, ., These, pieces, will, be, treated, much, like, the, bean, sprouts, ,, though, perhaps, with, slightly, more, co

In [0]:
all_entities

[[(9, 13, 'dish')],
 [(9, 13, 'dish'), (83, 87, 'dish'), (144, 148, 'dish')],
 [(9, 15, 'vegetable')],
 [(9, 17, 'vegetable'), (205, 217, 'vegetable'), (296, 302, 'vegetable')],
 [(9, 21, 'vegetable')],
 [],
 [(121, 128, 'dish')],
 [(8, 12, 'dish')],
 [(8, 17, 'condiment'),
  (28, 32, 'dish'),
  (66, 72, 'vegetable'),
  (77, 89, 'vegetable')],
 [(4, 10, 'condiment')],
 [(8, 12, 'dish'),
  (14, 26, 'vegetable'),
  (32, 37, 'vegetable'),
  (47, 51, 'dish')],
 [],
 [(235, 242, 'dish')],
 [],
 [(16, 21, 'vegetable'),
  (115, 124, 'condiment'),
  (145, 154, 'dish'),
  (158, 163, 'GPE')],
 [(8, 12, 'dish'), (100, 106, 'condiment')],
 [(143, 147, 'dish'), (155, 164, 'dish')],
 [(111, 115, 'dish'),
  (193, 197, 'dish'),
  (319, 321, 'CARDINAL'),
  (325, 327, 'CARDINAL'),
  (328, 335, 'TIME')],
 [(20, 24, 'dish'), (38, 42, 'dish'), (119, 123, 'dish'), (153, 160, 'dish')],
 [(128, 135, 'dish')],
 [],
 [(24, 30, 'TIME')],
 [(23, 29, 'condiment'), (86, 92, 'condiment'), (126, 132, 'condiment')],
 

In [0]:
to_train_ents = []
with open ('rice.txt') as train_file:
  doc = True
  i = 0
  while doc:
    doc = train_file.readline()
    entity = all_entities[i]
    i += 1
    to_train_ents.append((doc, dict(entities = entity)))



In [0]:
to_train_ents[0:5]

[('If using tofu, cook it.\n', {'entities': [(9, 13, 'dish')]}),
 ('Cook the rice as you normally would, but be careful not to use too much water. The rice should not be too sticky. It may help to let the cooked rice sit overnight in the refrigerator.\n',
  {'entities': [(9, 13, 'dish'), (83, 87, 'dish'), (144, 148, 'dish')]}),
 ('If using onions, dice them into whatever size you prefer.\n',
  {'entities': [(9, 15, 'vegetable')]}),
 ('If using bok choy, separate the stalk part from the leaf part. Slice the leaf part crosswise into 3/4-inch-thick strips, then tear the strips into 2-inch pieces. These pieces will be treated much like the bean sprouts, though perhaps with slightly more cooking. The stalk part will be treated as celery.\n',
  {'entities': [(9, 17, 'vegetable'),
    (205, 217, 'vegetable'),
    (296, 302, 'vegetable')]}),
 ('If using bean sprouts, wash them and discard any that are not white and crunchy.\n',
  {'entities': [(9, 21, 'vegetable')]})]

In [0]:
nlp = spacy.load("en_core_web_lg")
ruler = EntityRuler(nlp).from_disk("patterns.jsonl")
emoji = Emoji(nlp)
nlp.add_pipe(emoji, first=True)
nlp.add_pipe(ruler, before='ner')
nlp.pipe_names

['emoji', 'tagger', 'parser', 'entity_ruler', 'ner']

In [83]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in ["ner" , "entity_ruler"]]
other_pipes

['emoji', 'tagger', 'parser']

In [85]:
type(to_train_ents)

list

In [89]:
to_train_ents[0][1].get('entities')

[(9, 13, 'dish')]

In [93]:
model = 'en_core_web_lg'



"""Load the model, set up the pipeline and train the entity recognizer."""
if model is not None:
    nlp = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank("en")  # create blank Language class
    print("Created blank 'en' model")

# nlp.create_pipe works for built-ins that are registered with spaCy
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe("ner")

# add labels
for _, annotations in to_train_ents:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])



# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in ["ner" , "entity_ruler"]]
with nlp.disable_pipes(*other_pipes):  # only train NER
    # reset and initialize the weights randomly – but only if we're
    # training a new model
    if model is None:
        nlp.begin_training()



    for itn in range(100):
      random.shuffle(to_train_ents)
      losses = {}
# batch up the examples using spaCy's minibatch
      batches = minibatch(to_train_ents, size=compounding(4.0, 32.0, 1.001))

      for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(texts, annotations, drop=0.3, losses=losses)
        print(losses)

Loaded model 'en_core_web_lg'


ValueError: ignored

In [1]:
ner.labels

NameError: ignored

In [108]:
[tup for tup in to_train_ents if tup[0] == 'condimentU_spice']

[]

In [105]:
to_train_ents[5][1]['entities']

[(235, 242, 'dish')]

In [0]:
import re


def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])

    return cleaned_data

In [100]:
clean_train_ents = trim_entity_spans(to_train_ents)


TypeError: ignored

In [0]:
doc1 = nlp("I love fettuccine cheese 💚🌿 but hate rice, bok choy  and onion.")
print([(ent.text, ent.label_, ent.ent_id_) for ent in doc1.ents])

displacy.render(doc1,'ent',jupyter=True)

[('rice', 'dish', ''), ('bok choy', 'vegetable', ''), ('onion', 'vegetable', '')]


In [0]:
doc1 = nlp("I love fettuccine cheese 💚🌿 but hate rice, bok choy  and onion.")
print([(ent.text, ent.label_, ent.ent_id_) for ent in doc1.ents])

displacy.render(doc1,'ent',jupyter=True)


[('rice', 'dish', ''), ('bok', 'dish', ''), ('onion', 'dish', '')]


In [0]:
ner.labels

('GPE',
 'herb',
 'condiment',
 'ORDINAL',
 'PERCENT',
 'LAW',
 'LANGUAGE',
 'MONEY',
 'PERSON',
 'LOC',
 'fruit',
 'dish',
 'WORK_OF_ART',
 'CARDINAL',
 'bread',
 'spice',
 'ORG',
 'PRODUCT',
 'DATE',
 'TIME',
 'QUANTITY',
 'NORP',
 'EVENT',
 'FAC',
 'vegetable')

In [0]:
for _, annotations in to_train_ents:
    for ent in annotations.get("entities"):
        print(ent[2])

dish
dish
dish
dish
dish
condiment
dish
vegetable
vegetable
TIME
condiment
CARDINAL
dish
condiment
dish
ORDINAL
TIME
dish
dish
CARDINAL
condiment
dish
ORDINAL
ORDINAL
dish
dish
dish
dish
vegetable
vegetable
condiment
condiment
condiment
dish
dish
dish
dish
vegetable
vegetable
dish
dish
dish
dish
dish
dish
dish
dish
ORDINAL
dish
dish
dish
dish
condiment
dish
dish
vegetable
vegetable
vegetable
dish
dish
dish
CARDINAL
CARDINAL
TIME
spice
dish
dish
condiment
dish
ORDINAL
ORDINAL
TIME
condiment
vegetable
vegetable
vegetable
dish
condiment
dish
dish
dish
CARDINAL
TIME
dish
dish
dish
vegetable
vegetable
dish
dish
dish
dish
dish
vegetable
condiment
dish
GPE


In [0]:
check = np.random.randint(0,50000,10)
print(check)

for doc in list(nlp.pipe(texts[check])):
  displacy.render(doc,'ent',jupyter=True)
#  displacy.render(doc,'dep',jupyter=True,options={'distance': 60, 'fine_grained': True})

[48019 32965 14151 42060 46939 16072  2078 31236  4464 32608]
